In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.nn import init

In [3]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, matthews_corrcoef, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from Element_PI_JC import VariancePersist_JC
import glob

Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [4]:
# Set up constants
receptor = 'GCR'

# Bayesian TPI Hyperparameters 
pixel = 21
myspread = 0.06

# MLPC Epochs
num_epochs = 300

# Load SMILES strings
def load_smiles(file_path):
    with open(file_path, 'r') as f:
        return [line.strip() for line in f]

# Define MLPC Architecture
class MLPC(nn.Module):
    def __init__(self, input_size, seed=20):
        super(MLPC, self).__init__()
        torch.manual_seed(seed)
        self.model = nn.Sequential(
            nn.Linear(input_size, 300),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(300, 150),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(150, 75),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(75, 2),
        )
        self._initialize_weights(seed)

    def forward(self, x):
        return self.model(x)

    def _initialize_weights(self, seed):
        torch.manual_seed(seed)
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

# Generate Persistence Images
def generate_persistence_images(xyz_files, pixelx, pixely, spread, myspecs):
    TT_manual = []
    for cmp in xyz_files:
        TT_image = VariancePersist_JC(cmp, pixelx=pixelx, pixely=pixely, myspread=spread, myspecs=myspecs, showplot=False)
        TT_manual.append(TT_image.flatten())
    return np.array(TT_manual)

# Function to compute MACCS keys
def get_maccs_keys(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return np.array(AllChem.GetMACCSKeysFingerprint(mol))

# Load data
train_test_files = sorted(glob.glob(f'{receptor}_*.xyz'))
validation_files = sorted(glob.glob(f'Validation/{receptor}_*.xyz'))
train_test_labels_df = pd.read_excel(f'{receptor}_MLinput.xlsx', header=None)
validation_labels_df = pd.read_excel(f'Validation/{receptor}_Validation.xlsx', header=None)

labels_train_test = train_test_labels_df.iloc[:, 1].values
labels_validation = validation_labels_df.iloc[:, 1].values

train_test_smiles = load_smiles(f'{receptor}_TT_SMILES.smi')
validation_smiles = load_smiles(f'Validation/{receptor}_V_SMILES.smi')

def train_and_evaluate():
    # Load data
    pixelx, pixely, spread, myspecs = pixel, pixel, myspread, {"maxBD": 2.5, "minBD": -0.1}
    train_test_pi = generate_persistence_images(train_test_files, pixelx, pixely, myspread, myspecs)
    validation_pi = generate_persistence_images(validation_files, pixelx, pixely, myspread, myspecs)

    train_test_maccs = np.array([get_maccs_keys(smiles) for smiles in train_test_smiles])
    validation_maccs = np.array([get_maccs_keys(smiles) for smiles in validation_smiles])

    # Define scaler lists for grid search
    maccs_scalers = [0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00]
    pi_scalers = [0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00]
    # Initialize variables for grid search
    best_validation_mcc = -np.inf
    best_hyperparameters = None
    best_fold_metrics = []
    best_loss_data = []
    best_auc_data = []
    best_validation_metrics = {}

    for maccs_scaler in maccs_scalers:
        for pi_scaler in pi_scalers:
            #print(f"Testing MACCS scaler: {maccs_scaler}, PI scaler: {pi_scaler}")
            
            # Apply scalers to description vectors
            scaled_train_test_maccs = train_test_maccs * maccs_scaler
            scaled_validation_maccs = validation_maccs * maccs_scaler
            scaled_train_test_pi = train_test_pi * pi_scaler
            scaled_validation_pi = validation_pi * pi_scaler

            train_test_data = np.hstack([scaled_train_test_pi, scaled_train_test_maccs])
            validation_data = np.hstack([scaled_validation_pi, scaled_validation_maccs])

            skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            smote = SMOTE(random_state=42)

            fold_metrics = []
            loss_data = []
            auc_data = []

            for fold_idx, (train_idx, test_idx) in enumerate(skf.split(train_test_data, labels_train_test)):
                X_train, X_test = train_test_data[train_idx], train_test_data[test_idx]
                y_train, y_test = labels_train_test[train_idx], labels_train_test[test_idx]

                X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
                X_train_tensor = torch.tensor(X_train_smote, dtype=torch.float32)
                y_train_tensor = torch.tensor(y_train_smote, dtype=torch.long)
                X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
                y_test_tensor = torch.tensor(y_test, dtype=torch.long)

                model = MLPC(input_size=X_train_tensor.shape[1], seed=20)
                optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
                criterion = nn.CrossEntropyLoss()

                train_losses = []
                test_losses = []

                for epoch in range(num_epochs):
                    model.train()
                    optimizer.zero_grad()
                    train_outputs = model(X_train_tensor)
                    train_loss = criterion(train_outputs, y_train_tensor)
                    train_loss.backward()
                    optimizer.step()
                    train_losses.append(train_loss.item())

                    model.eval()
                    with torch.no_grad():
                        test_outputs = model(X_test_tensor)
                        test_loss = criterion(test_outputs, y_test_tensor)
                        test_losses.append(test_loss.item())

                loss_data.extend([{"Fold": fold_idx + 1, "Epoch": e + 1, "Dataset": "Training", "Loss": train_losses[e]} 
                                  for e in range(num_epochs)] +
                                 [{"Fold": fold_idx + 1, "Epoch": e + 1, "Dataset": "Testing", "Loss": test_losses[e]} 
                                  for e in range(num_epochs)])

                with torch.no_grad():
                    train_outputs = model(X_train_tensor)
                    train_preds = torch.argmax(train_outputs, dim=1).numpy()
                    train_probs = torch.softmax(train_outputs, dim=1)[:, 1].numpy()
                    train_acc = accuracy_score(y_train_smote, train_preds)
                    train_bal_acc = balanced_accuracy_score(y_train_smote, train_preds)
                    train_mcc = matthews_corrcoef(y_train_smote, train_preds)
                    train_auc = roc_auc_score(y_train_smote, train_probs)

                    test_outputs = model(X_test_tensor)
                    test_preds = torch.argmax(test_outputs, dim=1).numpy()
                    test_probs = torch.softmax(test_outputs, dim=1)[:, 1].numpy()
                    test_acc = accuracy_score(y_test, test_preds)
                    test_bal_acc = balanced_accuracy_score(y_test, test_preds)
                    test_mcc = matthews_corrcoef(y_test, test_preds)
                    test_auc = roc_auc_score(y_test, test_probs)

                fold_metrics.append({
                    "Fold": fold_idx + 1,
                    "Train Accuracy": train_acc, "Train Balanced Accuracy": train_bal_acc,
                    "Train MCC": train_mcc, "Train AUC": train_auc,
                    "Test Accuracy": test_acc, "Test Balanced Accuracy": test_bal_acc,
                    "Test MCC": test_mcc, "Test AUC": test_auc
                })

                fpr_train, tpr_train, thresholds_train = roc_curve(y_train_smote, train_probs)
                fpr_test, tpr_test, thresholds_test = roc_curve(y_test, test_probs)
                auc_data.extend([
                    {"Fold": fold_idx + 1, "Dataset": "Training", "FPR": f, "TPR": t, "Threshold": th}
                    for f, t, th in zip(fpr_train, tpr_train, thresholds_train)
                ] + [
                    {"Fold": fold_idx + 1, "Dataset": "Testing", "FPR": f, "TPR": t, "Threshold": th}
                    for f, t, th in zip(fpr_test, tpr_test, thresholds_test)
                ])

            validation_tensor = torch.tensor(validation_data, dtype=torch.float32)
            with torch.no_grad():
                validation_outputs = model(validation_tensor)
                validation_preds = torch.argmax(validation_outputs, dim=1).numpy()
                validation_probs = torch.softmax(validation_outputs, dim=1)[:, 1].numpy()
                val_acc = accuracy_score(labels_validation, validation_preds)
                val_bal_acc = balanced_accuracy_score(labels_validation, validation_preds)
                val_mcc = matthews_corrcoef(labels_validation, validation_preds)
                val_auc = roc_auc_score(labels_validation, validation_probs)

                validation_metrics = {
                    "Accuracy": val_acc, "Balanced Accuracy": val_bal_acc,
                    "MCC": val_mcc, "AUC": val_auc
                }

            if validation_metrics["MCC"] > best_validation_mcc:
                best_validation_mcc = validation_metrics["MCC"]
                best_hyperparameters = {"MACCS Scaler": maccs_scaler, "PI Scaler": pi_scaler}
                best_fold_metrics = fold_metrics
                best_loss_data = loss_data
                best_auc_data = auc_data
                best_validation_metrics = validation_metrics

    # Save results to CSV files
    pd.DataFrame(best_fold_metrics).to_csv(f"{receptor}_TSV_best_fold_metrics.csv", index=False)
    pd.DataFrame(best_loss_data).to_csv(f"{receptor}_TSV_best_loss_data.csv", index=False)
    pd.DataFrame(best_auc_data).to_csv(f"{receptor}_TSV_best_auc_data.csv", index=False)
    pd.DataFrame([best_hyperparameters]).to_csv(f"{receptor}_TSV_best_hyperparameters.csv", index=False)
    pd.DataFrame([best_validation_metrics]).to_csv(f"{receptor}_TSV_best_validation_metrics.csv", index=False)

    print("Best model results have been saved.")

train_and_evaluate()

Best model results have been saved.
